# FindingModel Index


In [ ]:
from pprint import pprint

from findingmodel import ChoiceAttributeIded, ChoiceValueIded, FindingModelFull, Index, NumericAttributeIded

`Index` is a list of the basic metadata about finding models. It is stored in a DuckDB database file,
configured via the `duckdb_index_path` setting (defaults to `finding_models.duckdb` in the
platform-native user data directory).

The index can be populated from a directory of `*.fm.json` files containing model definitions,
such as at the [findingmodels repository](https://github.com/openimagingdata/findingmodels).

## Read Operations

For querying the index, simply create a read-only Index instance. No setup is required - the connection is established automatically on first use.


In [ ]:
# Create a read-only index (default mode)
index = Index(read_only=True)

In [3]:
await index.count()

1949

### Get Operations

Get an entry from the index (metadata only; use `get_full()` to actually get the finding model) using ID, name, or one of its synonyms.


In [ ]:
entry = await index.get("abdominal aortic aneurysm")
assert entry is not None
print(entry.model_dump_json(indent=2))

{
  "oifm_id": "OIFM_MSFT_134126",
  "name": "abdominal aortic aneurysm",
  "slug_name": "abdominal_aortic_aneurysm",
  "filename": "abdominal_aortic_aneurysm.fm.json",
  "file_hash_sha256": "36fe838b81f63a5d5da6b1c1a2a30a1900444db36da3c9870f8a4f6276a5b6ec",
  "description": "An abdominal aortic aneurysm (AAA) is a localized dilation of the abdominal aorta, typically defined as a diameter greater than 3 cm, which can lead to rupture and significant morbidity or mortality.",
  "synonyms": [
    "AAA"
  ],
  "tags": null,
  "contributors": [
    "HeatherChase"
  ],
  "attributes": [
    {
      "attribute_id": "OIFMA_MSFT_898601",
      "name": "presence",
      "type": "choice"
    },
    {
      "attribute_id": "OIFMA_MSFT_783072",
      "name": "change from prior",
      "type": "choice"
    }
  ]
}


### Get Full Model

Use `get_full()` to retrieve the complete FindingModelFull object, not just the metadata:


In [ ]:
# Get the complete model object
full_model = await index.get_full("abdominal aortic aneurysm")
assert full_model is not None

print(f"Model: {full_model.name}")
print(f"Description: {full_model.description}")
print(f"Number of attributes: {len(full_model.attributes)}")
print(f"\nFirst attribute: {full_model.attributes[0].name}")
if hasattr(full_model.attributes[0], "values"):
    print(f"  Values: {[v.name for v in full_model.attributes[0].values]}")

### Search Operations

Use `search()` to find models by name, synonyms, or description. The search supports fuzzy matching and is useful for discovering models or checking for potential duplicates:


In [ ]:
# Search for models related to "abdominal"
results = await index.search("abdominal")
print(f"Found {len(results)} results for 'abdominal'")
for i, result in enumerate(results[:3]):  # Show first 3
    print(f"\n{i + 1}. {result.name} (ID: {result.oifm_id})")
    print(f"   Description: {result.description[:80]}...")

## Updating the Index

The following sections show how to modify the index by adding, updating, or removing models. These operations require a writable index.


In [ ]:
# Create a writable index and ensure schema is ready
index = Index()  # read_only=False by default
await index.setup()

### Add/Remove Model

Note that adding a model performs a number of checks, especially for duplicate IDs, duplicated names, duplicate synonyms.


In [5]:
new_model = FindingModelFull(
    oifm_id="OIFM_TEST_123456",
    name="Test Model",
    description="A simple test finding model.",
    synonyms=["Test Synonym"],
    tags=["tag1", "tag2"],
    attributes=[
        ChoiceAttributeIded(
            oifma_id="OIFMA_TEST_123456",
            name="Severity",
            description="How severe is the finding?",
            values=[
                ChoiceValueIded(value_code="OIFMA_TEST_123456.0", name="Mild"),
                ChoiceValueIded(value_code="OIFMA_TEST_123456.1", name="Severe"),
            ],
            required=True,
            max_selected=1,
        ),
        NumericAttributeIded(
            oifma_id="OIFMA_TEST_654321",
            name="Size",
            description="Size of the finding.",
            minimum=1,
            maximum=10,
            unit="cm",
            required=False,
        ),
    ],
)
with open("test_model.fm.json", "w") as f:
    f.write(new_model.model_dump_json(indent=2))

In [6]:
await index.add_or_update_entry_from_file("test_model.fm.json")

<IndexReturnType.ADDED: 'added'>

In [7]:
await index.count()

1950

In [8]:
await index.remove_entry("Test Model")  # Can either use name or OIFM ID
await index.count()

1949

### Synchronize with a Definition Directory

You can use `update_from_directory()` to update the state of the index from the definition files in
directory.


In [12]:
from pathlib import Path

added, updated, removed = await index.update_from_directory(Path.cwd().parent.parent / "findingmodels" / "defs")

In [13]:
print(added, updated, removed)

0 0 0


In [14]:
await index.count()

1949